In [2]:
import nest_asyncio
nest_asyncio.apply()
# EDIT: the __import__ line is not needed, leaving it just 
# so that the comments make sense
# __import__('IPython').embed()

#### Test 101

In [3]:
from agents import Agent, Runner
from dotenv import  load_dotenv
from agents import set_default_openai_api,set_default_openai_key 
import os

load_dotenv("D:\\Code\\AI\\.env")

set_default_openai_api("chat_completions")
openai_api_key = os.environ.get("OPENAI_API_KEY")
set_default_openai_key(openai_api_key)

agent = Agent(name="Assistant", instructions="You are a helpful assistant",model='gpt-4o-mini')

result = Runner.run_sync(agent, "Write a haiku about recursion in programming.",max_turns=2)
print(result.final_output)

# Code within the code,
# Functions calling themselves,
# Infinite loop's dance.

Functions call themselves,  
Loops within loops, tracing deep,  
Endless paths unfold.


Role: system, user, assistant and developer(new)

# 5. Responses API

In [5]:
load_dotenv("D:\\Code\\AI\\.env")
from openai import OpenAI
client = OpenAI()

#### 5.1 Introduction


https://platform.openai.com/docs/api-reference/responses


In [ ]:
# --------------------------------------------------------------
# Basic text example with the Chat Completions API
# --------------------------------------------------------------

from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(model='gpt-4o-mini',
                                          messages=[{
                                              "role": "user",
                                              'content':"write a 2 liner joke on cars"
                                          }
                                              
                                          ])
print(response.choices[0].message.content)

In [ ]:
# --------------------------------------------------------------
# Basic text example with the Responses API
# --------------------------------------------------------------

response = client.responses.create(model='gpt-4o-mini',
                                   input = [
                                       {'role':'user','content':'What is present in the image?'},
                                       {'role':'user',
                                        'content': [{
                                            'type': 'input_image',
                                            'image_url': "https://upload.wikimedia.org/wikipedia/commons/3/3b/LeBron_James_Layup_%28Cleveland_vs_Brooklyn_2018%29.jpg",
                                        }] }
                                   ])
print(response.output_text)

In [ ]:
# --------------------------------------------------------------
# Streaming
# --------------------------------------------------------------

stream = client.responses.create(model='gpt-4o-mini',
                                input="write a 2 liner joke on cars",
                            stream=True  )

# Store chunks in a list

text_chunks=[]

for event in stream:
    if hasattr(event,"type") and "text.delta" in event.type:
        text_chunks.append(event.delta)
        print(event.delta,end="",flush=True)

#### 5.2 Text Prompting

Model spec: https://model-spec.openai.com/2025-02-12.html

Dashboard: https://platform.openai.com/logs?api=responses

Inputs can now be a single string or a list of messages.

The list of roles can now be:
- system
- developer
- user
- assistant

In [ ]:
# --------------------------------------------------------------
# Introducing instructions
# --------------------------------------------------------------

response = client.responses.create(model="gpt-4o-mini",
                                   instructions="Talk like Caption Jack Sparrow.",
                                   input=" Tell me a 2 liner joke on ships or seas.")

print(response.output_text)

In [ ]:
# --------------------------------------------------------------
# Which would be similar to:
# --------------------------------------------------------------
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "developer", "content": "Talk like Caption Jack Sparrow."},
        {"role": "user", "content": "Tell me a 2 liner joke on ships or seas."},
    ],
)

print(response.output_text)


Here is the ordering of authority levels. Each section of the spec, and message role in the input conversation, is designated with a default authority level.

- Platform: Model Spec "platform" sections and system messages
- Developer: Model Spec "developer" sections and developer messages
- User: Model Spec "user" sections and user messages
- Guideline: Model Spec "guideline" sections
- No Authority: assistant and tool messages; quoted/untrusted text and multimodal data in other messages


https://model-spec.openai.com/2025-02-12.html#chain_of_command

In [ ]:
# --------------------------------------------------------------
# The chain of command (hierarchical instructions)
# --------------------------------------------------------------
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "system", "content": "Talk like Caption Jack Sparrow."},
        {"role": "developer", "content": "don't Talk like Caption Jack Sparrow."},
        {"role": "user", "content": "Tell me a 2 liner joke on ships or seas."},
    ],
)

print(response.output_text)  # talks like a pirate

In [ ]:
response = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "system", "content": "Don't talk like Caption Jack Sparrow."},
        {"role": "developer", "content": "Talk like Caption Jack Sparrow."},
        {"role": "user", "content": "Tell me a 2 liner joke on ships or seas."},
    ],
)

print(response.output_text)  # doesn't talk like a pirate

#### 5.3 Conversation State

https://platform.openai.com/docs/guides/conversation-state?api-mode=responses

In [6]:
# --------------------------------------------------------------
# Manual conversation state
# --------------------------------------------------------------

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "user", "content": "knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
)

print(response.output_text)

Orange who?


In [7]:
# --------------------------------------------------------------
# Dynamic conversation state
# --------------------------------------------------------------
history = [{"role": "user", "content": "tell me a joke"}]

response = client.responses.create(model="gpt-4o-mini", input=history, store=False)

print(response.output_text)


# Add the response to the conversation
history += [
    {"role": output.role, "content": output.content} for output in response.output
]

history.append({"role": "user", "content": "tell me another"})


second_response = client.responses.create(
    model="gpt-4o-mini", input=history, store=False
)

print(second_response.output_text)

Why don't scientists trust atoms?

Because they make up everything!
Why did the scarecrow win an award?

Because he was outstanding in his field!


In [ ]:
# --------------------------------------------------------------
# OpenAI APIs for conversation state (default is to store)
# --------------------------------------------------------------

response = client.responses.create(
    model="gpt-4o-mini",
    input="tell me a joke",
    # store=True It is true by defaults and stored by OpenAI

)
print(response.output_text)

second_response = client.responses.create(
    model="gpt-4o-mini",
    previous_response_id=response.id,
    input=[{"role": "user", "content": "explain why this is funny."}],
)
print(second_response.output_text)

Why did the scarecrow win an award?

Because he was outstanding in his field!
The joke is funny due to a clever play on words. "Outstanding" has a double meaning: it can mean exceptionally good, which implies the scarecrow is impressive, and it can also mean literally standing out in a field, which is what a scarecrow does. This pun creates a humorous twist, as the punchline connects the unexpected meaning with the situation of the scarecrow. The surprise and wordplay are key elements that make the joke work.


#### 5.4 Function Calling

In [15]:
from openai import OpenAI
client=OpenAI()

tools =[
    {
        "type": "function",
        "name": "send_email",
        "description": " Send an email to a given recipient with a subject and message.",
        "parameters": {
            'type': "object",
            "properties":{
            "to" : {"type": "string","description":"The recipient email address."},
            "subject":{"type":"string","description":"Email subject line."},
            "body":{"type":"string","description":"Body of the email message."},
                        },
            'required':["to","subject","body"],
            "additionalProperties":False,
        },
    }
]

response = client.responses.create(model='gpt-4o-mini',
                                   input="Can you send an email to chakri@yxm.com saying you are selected.",
                                   tools = tools)

print(response.output,end="\n \n")
print(response.output[0].model_dump_json(indent=2),end='\n \n')
# print(response.output[1].model_dump_json(indent=2),end='\n \n')

[ResponseFunctionToolCall(id='fc_67d6e0cde6788191af69774ba611ee0c01a25152cbf2c95a', arguments='{"to":"chakri@yxm.com","subject":"Congratulations!","body":"You have been selected."}', call_id='call_otTLztNAA3VW2lqU6IEgQ4Ss', name='send_email', type='function_call', status='completed')]
 
{
  "id": "fc_67d6e0cde6788191af69774ba611ee0c01a25152cbf2c95a",
  "arguments": "{\"to\":\"chakri@yxm.com\",\"subject\":\"Congratulations!\",\"body\":\"You have been selected.\"}",
  "call_id": "call_otTLztNAA3VW2lqU6IEgQ4Ss",
  "name": "send_email",
  "type": "function_call",
  "status": "completed"
}
 


In [14]:
from openai import OpenAI

client = OpenAI()

tools = [
    {
        "type": "function",
        "name": "send_email",
        "description": "Send an email to a given recipient with a subject and message.",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {"type": "string", "description": "The recipient email address."},
                "subject": {"type": "string", "description": "Email subject line."},
                "body": {"type": "string", "description": "Body of the email message."},
            },
            "required": ["to", "subject", "body"],
            "additionalProperties": False,
        },
    }
]

response = client.responses.create(
    model="gpt-4o",
    input=[{'role':'user','content': "Can you send an email to chakri@yxm.com and jay_roy@yxm.com saying you are selected."}],
    tools=tools)

print(response.output,end="\n \n")
print(response.output[0].model_dump_json(indent=2),end='\n \n')
print(response.output[1].model_dump_json(indent=2),end='\n \n')


[ResponseFunctionToolCall(id='fc_67d6e0a7f67c819199582056805e7b3002189529e264aefc', arguments='{"to":"chakri@yxm.com","subject":"Selection Notification","body":"Congratulations! You are selected."}', call_id='call_qxefE6GTLD1YMe78rGtt78IT', name='send_email', type='function_call', status='completed'), ResponseFunctionToolCall(id='fc_67d6e0a8b0cc81918bd951a88a0fe78c02189529e264aefc', arguments='{"to":"jay_roy@yxm.com","subject":"Selection Notification","body":"Congratulations! You are selected."}', call_id='call_nqsCuoMy4WLMOQkPTCqoer4l', name='send_email', type='function_call', status='completed')]
 
{
  "id": "fc_67d6e0a7f67c819199582056805e7b3002189529e264aefc",
  "arguments": "{\"to\":\"chakri@yxm.com\",\"subject\":\"Selection Notification\",\"body\":\"Congratulations! You are selected.\"}",
  "call_id": "call_qxefE6GTLD1YMe78rGtt78IT",
  "name": "send_email",
  "type": "function_call",
  "status": "completed"
}
 
{
  "id": "fc_67d6e0a8b0cc81918bd951a88a0fe78c02189529e264aefc",
  "a

#### 5.5 Structured Output

In [16]:
import json
from typing import List
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()


In [18]:
# --------------------------------------------------------------
# Using a JSON Schema
# --------------------------------------------------------------

response = client.responses.create(
    model= 'gpt-4o-mini',
    input=[
        {'role':'system','content':'Extract the event information.'},
        {'role':'user','content':'Karthik and Chakri are going to a AI Fair on Sunday.'},
    ],
    text={
        "format": {
            "type":"json_schema",
            "name": "calender_event",
            "schema": {
                "type":"object",
                "properties":{
                    "name":{"type":"string"},
                    "date":{"type":"string"},
                    "participants":{"type":"array","items":{"type":"string"}},
                },
                "required": ['name','date','participants'],
                "additionalProperties":False,
            },
            "strict": True,
        }
    }
)

event = json.loads(response.output_text)


In [19]:
event

{'name': 'AI Fair', 'date': 'Sunday', 'participants': ['Karthik', 'Chakri']}

https://news.ycombinator.com/item?id=43336251

In [17]:
# --------------------------------------------------------------
# Using a Pydantic model (and simple response format)
# --------------------------------------------------------------
class CalanderEvent(BaseModel):
    name : str
    date : str
    participants: List[str]


response = client.responses.parse(model='gpt-4o-mini',
                                  input='Karthik and Chakri are going to a AI Fair on Sunday.',
                                  instructions="Extract the event information",
                                  text_format=CalanderEvent)

response_model = response.output[0].content[0].parsed

print(type(response_model))
print(response_model.model_dump_json(indent=2))

<class '__main__.CalanderEvent'>
{
  "name": "AI Fair",
  "date": "Sunday",
  "participants": [
    "Karthik",
    "Chakri"
  ]
}


#### 5.6 Web Search

In [20]:
from openai import OpenAI

client = OpenAI()



In [22]:
# --------------------------------------------------------------
# Basic web search
# --------------------------------------------------------------

response = client.responses.create(model='gpt-4o-mini',
                                   tools=[{
                                       "type":"web_search_preview"
                                   }],
                                   input=" Who is Prahas from Tollywood ?")

print(response.output_text)

Prabhas, born Venkata Satyanarayana Prabhas Raju Uppalapati on October 23, 1979, in Chennai, India, is a prominent actor in the Telugu film industry, commonly known as Tollywood. He hails from a film-oriented family; his father, Uppalapati Surya Narayana Raju, is a renowned film producer in Telugu cinema. Prabhas completed his schooling at DNR School in Bhimavaram and pursued engineering at Sri Chaitanya College in Hyderabad. ([asianlifestylemagazine.com](https://asianlifestylemagazine.com/the-rise-of-prabhas-from-tollywood-star-to-global-icon/?utm_source=openai))

He made his acting debut in 2002 with the film "Eeswar," but it was his role in "Varsham" (2004) that established him as a leading actor in Tollywood. His career reached new heights with the epic fantasy film series "Baahubali," directed by S.S. Rajamouli. Prabhas portrayed the dual roles of Amarendra Baahubali and Mahendra Baahubali in "Baahubali: The Beginning" (2015) and "Baahubali 2: The Conclusion" (2017), which were mo

In [24]:
# --------------------------------------------------------------
# Basic web search with location
# --------------------------------------------------------------

response = client.responses.create(
    model="gpt-4o",
    tools=[
        {
            "type": "web_search_preview",
            "user_location": {
                "type": "approximate",
                "country": "IN",
                "city": "Hyderabad",
            },
        }
    ],
    input="What are the best biryani restaurants around hyderabad?",
)

print(response.output_text)
response.output[1].content[0].annotations
response.output[1].content[0].annotations[0].url

Hyderabad, renowned for its rich culinary heritage, offers a plethora of establishments serving exceptional biryani. Here are some of the top-rated biryani restaurants in the city:

**[Bawarchi](https://www.google.com/maps/search/Bawarchi%2C+Hyderabad%2C+India)**
_Hyderabad, India_
Located at RTC Cross Road, Bawarchi is famed for its authentic Hyderabadi dum biryani, offering both chicken and mutton varieties that are juicy and flavorful. Note: The original outlet has no branches.

**[Paradise](https://www.google.com/maps/search/Paradise%2C+Hyderabad%2C+India)**
_Hyderabad, India_
A culinary landmark since 1953, Paradise serves a delicately spiced biryani with tender meat, making it a must-visit for both locals and tourists.

**[Hotel Shadab](https://www.google.com/maps/search/Hotel+Shadab%2C+Hyderabad%2C+India)**
_Hyderabad, India_
Situated near Charminar, Hotel Shadab is renowned for its spicy biryani, offering both mutton and chicken options, along with other Hyderabadi delicacies.


'https://www.google.com/maps/search/Bawarchi%2C+Hyderabad%2C+India'

#### 5.7 File Search

https://platform.openai.com/storage/files/

In [ ]:
import requests
from io import BytesIO
from openai import OpenAI
import textwrap

client = OpenAI()


In [ ]:
# --------------------------------------------------------------
# Upload a file
# --------------------------------------------------------------


def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(file=file_tuple, purpose="assistants")
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(file=file_content, purpose="assistants")
    print(result.id)
    return result.id


# Replace with your own file path or URL
file_id = create_file(client, "https://cdn.openai.com/API/docs/deep_research_blog.pdf")

https://platform.openai.com/storage/vector_stores

Please be aware of costs!

In [ ]:
# --------------------------------------------------------------
# Create a vector store
# --------------------------------------------------------------


vector_store = client.vector_stores.create(name="knowledge_base")
print(vector_store.id)

In [ ]:
# --------------------------------------------------------------
# Add a file to the vector store
# --------------------------------------------------------------

result = client.vector_stores.files.create(
    vector_store_id=vector_store.id, file_id=file_id
)
print(result)

In [ ]:
# --------------------------------------------------------------
# Check status
# --------------------------------------------------------------

result = client.vector_stores.files.list(vector_store_id=vector_store.id)
print(result)

In [ ]:
# --------------------------------------------------------------
# Use file search
# --------------------------------------------------------------

"""
At the moment, you can search in only one vector store at a time, 
so you can include only one vector store ID when calling the file search tool.
"""

response = client.responses.create(
    model="gpt-4o",
    input="What is deep research by OpenAI?",
    tools=[{"type": "file_search", "vector_store_ids": [vector_store.id]}],
)
print(response)
print(textwrap.fill(response.output_text, width=80))

In [ ]:

# --------------------------------------------------------------
# Limit results
# --------------------------------------------------------------

response = client.responses.create(
    model="gpt-4o",
    input="What is deep research by OpenAI?",
    tools=[
        {
            "type": "file_search",
            "vector_store_ids": [vector_store.id],
            "max_num_results": 2,
        }
    ],
    include=["output[*].file_search_call.search_results"],
)
print(response.model_dump_json(indent=2))

In [ ]:
# --------------------------------------------------------------
# Similarity search
# ----------------------§---------------------------------------


results = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query="What is deep research by OpenAI?",
)

print(results.model_dump_json(indent=2))

#### 5.8  Reasoning

https://platform.openai.com/docs/guides/reasoning?api-mode=responses

In [ ]:
from openai import OpenAI

client = OpenAI()


prompt = """
Write a bash script that takes a matrix represented as a string with 
format '[1,2],[3,4],[5,6]' and prints the transpose in the same format.
"""

response = client.responses.create(
    model="o3-mini",
    reasoning={"effort": "medium"}, # <-----
    input=[{"role": "user", "content": prompt}],
)

print(response.output_text)

BadRequestError: Error code: 400 - {'error': {'message': "Unsupported parameter: 'reasoning.effort' is not supported with this model.", 'type': 'invalid_request_error', 'param': 'reasoning.effort', 'code': 'unsupported_parameter'}}